In [1]:
import argparse
import os

import numpy as np

import shutil
import time

from tqdm import tqdm

import wandb

import sys
import time

import cv2
import json

util_path = '/cluster/home/jpostels/nnaimi/instant-ngp/scripts'
sys.path.append(util_path)

from common import *
from scenes import *

pyngp_path = '/cluster/home/jpostels/nnaimi/instant-ngp/build'
sys.path.append(pyngp_path)

import pyngp as ngp # noqa

In [2]:
## PATHS
DATASET_PATH= '/cluster/work/cvl/jpostels/nnaimi/data'
OUTPUT_PATH= '/cluster/work/cvl/jpostels/nnaimi/output'
CONFIG_PATH= '/cluster/home/jpostels/nnaimi/instant-ngp/configs/image'

In [3]:
name = 'kodim11'
train = True
n_steps=50000
width=768
height=512
screenshot_spp=1
log_imgs=True
log_freq=1000
scene = DATASET_PATH + '/KODAK/kodim11.png'
network = CONFIG_PATH + '/notebook.json'
screenshot_dir = OUTPUT_PATH
save_snapshot = OUTPUT_PATH
load_snapshot=""

wandb_tags = ["exploration", "varying-scaling-parameter-2"]

In [4]:
def get_scene(scene):
    for scenes in [scenes_sdf, scenes_nerf, scenes_image, scenes_volume]:
        if scene in scenes:
            return scenes[scene]
    return None

In [5]:
!printenv | grep ROOT_DIR

In [6]:
testbed = ngp.Testbed()
testbed.root_dir = ROOT_DIR

# Open config JSON file and load its contents into a dictionary
with open(network, 'r') as json_file:
    config = json.load(json_file)

18:47:24 SUCCESS  Initialized CUDA. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]


In [7]:
# Load the PNG image (converts the image from srgb to linear np array)
groundtruth = read_image(scene)
print("Groundtruth shape: ", groundtruth.shape)

if scene:
    scene_info = get_scene(scene)
    if scene_info is not None:
        scene = os.path.join(scene_info["data_dir"], scene_info["dataset"])
        if not network and "network" in scene_info:
            network = scene_info["network"]

    testbed.load_training_data(scene)

if load_snapshot:
    scene_info = get_scene(load_snapshot)
    if scene_info is not None:
        load_snapshot = default_snapshot_filename(scene_info)
    testbed.load_snapshot(load_snapshot)
elif network:
    testbed.reload_network_from_file(network)

testbed.shall_train = train

config["logging_frequency"] = log_freq
config["logging_images"] = log_imgs
config["name"] = name
config["n_enc_params"] = testbed.n_encoding_params()
config["n_mlp_params"] = testbed.n_params() - testbed.n_encoding_params()
config["n_params"] = testbed.n_params()

wandb.init(
    project="instantNGP-4-Compression",
    name=name,
    dir="/cluster/work/cvl/jpostels/nnaimi/wandb",
    config=config,
    tags=wandb_tags
)

network_stem = os.path.splitext(os.path.basename(network))[0] if network else "base"

old_training_step = 0
tqdm_last_update = 0
start = time.monotonic_ns()
log_dict = {}

if n_steps > 0:
    with tqdm(desc="Training", total=n_steps, unit="steps") as t:

        while testbed.frame():
            if testbed.want_repl():
                repl(testbed)

            # What will happen when training is done?
            if testbed.training_step >= n_steps:
                break

            # Update progress bar
            if testbed.training_step < old_training_step or old_training_step == 0:
                old_training_step = 0
                t.reset()

            # logging
            if not testbed.training_step % log_freq:
                psnr = -10.0 * np.log(testbed.loss) / np.log(10.0)
                elapsed_time = (time.monotonic_ns() - start) * 1e-9

                if log_imgs and screenshot_dir:
                    # log images and training info
                    outname = os.path.join(screenshot_dir, scene + "_" + network_stem)
                    image = testbed.render(width or 1920, height or 1080, screenshot_spp, True)


                    assert image[:,:,:3].shape == groundtruth.shape, "image and groundtruth shapes not equal" 
                    residual = groundtruth - image[:,:,:3]
                    res_mean = np.mean(residual)

                    if os.path.dirname(outname) != "":
                        os.makedirs(os.path.dirname(outname), exist_ok=True)

                    write_image(outname + ".png", image)

                    image = linear_to_srgb(image) 
                    residual = linear_to_srgb(residual)

                    log_dict = {"loss": np.log(testbed.loss) / np.log(10.0), 
                                "training_step": testbed.training_step, 
                                "psnr": psnr,
                                "elapsed_time": elapsed_time,
                                "encoded_image": wandb.Image(image),
                                "residual": wandb.Image(residual),
                                "res_mean": res_mean}
                else:
                    # log training info only
                    log_dict = {"loss": np.log(testbed.loss) / np.log(10.0), 
                                "training_step": testbed.training_step, 
                                "psnr": psnr,
                                "elapsed_time": elapsed_time}

                # WandB logging
                wandb.log(log_dict)

            # update tqdm
            now = time.monotonic()
            if now - tqdm_last_update > 0.1:
                t.update(testbed.training_step - old_training_step)
                t.set_postfix(loss=testbed.loss)
                old_training_step = testbed.training_step
                tqdm_last_update = now

# Total training time
elapsed_time = (time.monotonic_ns() - start) * 1e-9	

if save_snapshot:
    outfilename = os.path.splitext(os.path.split(scene)[1])[0] + "-" + time.strftime("%Y%m%d-%H%M%S")
    testbed.save_snapshot(save_snapshot + "/" + outfilename + ".ingp", False)

image = None
if screenshot_dir:
    outfilename = os.path.splitext(os.path.split(scene)[1])[0]
    outname = os.path.join(screenshot_dir, outfilename + "_" + network_stem)
    print(f"Rendering {outname}.png")
    image = testbed.render(width or 1920, height or 1080, screenshot_spp, True)
    if os.path.dirname(outname) != "":
        os.makedirs(os.path.dirname(outname), exist_ok=True)
    write_image(outname + ".png", image)

residual = groundtruth - image[:, :, :3]

res_mean = np.mean(residual)

res_mean_r = np.mean(residual[:,:,0])
res_mean_g = np.mean(residual[:,:,1])
res_mean_b = np.mean(residual[:,:,2])

image = linear_to_srgb(image)
residual = linear_to_srgb(residual)

psnr = -10.0 * np.log(testbed.loss) / np.log(10.0)

wandb.log({"loss": np.log(testbed.loss) / np.log(10.0),
            "psnr": psnr,
            "training_step": testbed.training_step, 
            "elapsed_time": elapsed_time,
            "encoded_image": wandb.Image(image),
            "residual": wandb.Image(residual),
            "res_mean": res_mean,
            "res_mean_r": res_mean_r,
            "res_mean_g": res_mean_g,
            "res_mean_b": res_mean_b
            })

wandb.finish()

image is srgb
Groundtruth shape:  (512, 768, 3)
18:47:24 INFO     Loading STBI image from /cluster/work/cvl/jpostels/nnaimi/data/KODAK/kodim11.png
18:47:24 SUCCESS  Loaded a full-precision image with 768x512 pixels.
18:47:24 INFO     Loading network config from: /cluster/home/jpostels/nnaimi/instant-ngp/configs/image/notebook.json
18:47:24 INFO     GridEncoding:  Nmin=64 b=1.2 F=2 T=2^12 L=4
18:47:24 INFO     Model:         2--[HashGrid]-->16--[FullyFusedMLP(neurons=64,layers=4)]-->3
18:47:24 INFO       total_encoding_params=32768 total_network_params=6144


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nasibnaimi (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Training: 100%|█████████▉| 49923/50000 [01:51<00:00, 448.16steps/s, loss=0.00147]


18:50:25 SUCCESS  Saved snapshot '/cluster/work/cvl/jpostels/nnaimi/output/kodim11-20230719-185025.ingp'
Rendering /cluster/work/cvl/jpostels/nnaimi/output/kodim11_notebook.png


elapsed_time,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████████
res_mean,▄▆▄▆▃▇▂▇▇▆▇▁▆▄▄█▆▅▇▅▆▅▆▅▅▆▅▅▅▅▄▅▅▅▅▅▅▅▅▅
res_mean_b,▁
res_mean_g,▁
res_mean_r,▁
training_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
elapsed_time,111.43723
loss,-2.8347
psnr,28.34702
